In [1]:
import pandas as pd
import re
import string

In [49]:
scraped= pd.read_csv('Hotel Location Table Mapping.csv',encoding='latin1')
hotels=pd.read_csv('Hotel Location.csv',encoding='latin1')
old=pd.read_csv('old.csv',encoding='latin1')
pivot=pd.read_csv('pivot.csv',encoding='latin1')

In [50]:
scraped.head()

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped
0,https://www.flightcentre.co.za/product/14197346,flightcentre,maldives,Adaaran Select Meedhupparu,\nYour Maldives Holiday Package includes:\nRet...,\nYour Maldives Holiday Package includes:\n\nR...,NaN,NaN
1,https://www.flightcentre.co.za/product/5415249,flightcentre,south africa,Family Fun on the Wild Coast,\nYour Eastern Cape Holiday Package includes:\...,\nYour Eastern Cape Holiday Package includes:\...,NaN,NaN
2,https://www.flightcentre.co.za/product/16385011,flightcentre,mauritius,Azuri Residences by Life in Blue,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN
3,https://www.flightcentre.co.za/product/16323810,flightcentre,mauritius,Seas the Day in Mauritius,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN
4,https://www.flightcentre.co.za/product/13250621,flightcentre,south africa,Kruger Park Splendour,\nYour Kruger National Park Holiday Package in...,\nYour Kruger National Park Holiday Package in...,NaN,NaN


In [51]:
hotels.head()

,Name,Address1,Address2,City,StateProvince,PostalCode,Country,Latitude,Longitude,AirportCode,PropertyCurrency,StarRating,Location
0,41,41 Buckingham Palace Road,NaN,London,NaN,SW1W 0PS,GB,51.49830,-0.14372,LHR,GBP,5.0,Near Buckingham Palace
1,#2 Andrews  Port Douglas,#2 Andrews Close,NaN,Port Douglas,QL,4877,AU,-16.51689,145.46770,CNS,AUD,5.0,Near Four Mile Beach Park
2,#2 Hastings Street,2 Hastings Street,NaN,Noosa Heads,QL,4567,AU,-26.38616,153.08722,MCY,AUD,4.0,In Noosa Heads (Noosa)
3,.Com,Albena,NaN,Albena,NaN,9620,BG,43.37024,28.07389,VAR,EUR,3.0,In Albena
4,?The Dunstane,4 West Coates & 5 Hampton Terrace,Haymarket,Edinburgh,NaN,EH12 5JQ,GB,55.94571,-3.22880,EDI,GBP,4.0,Near Murrayfield Stadium


In [3]:
def clean_text(text):
    """
    This function uses regular expressions to 
        - remove links characters
        - remove html characters,
        - remove escape sequences,
        - remove capitalization,
        - any extra white space from each text and then converts them to lowercase.

    Input:
    text: original text
          datatype: string

    Output:
    texts: modified text
           datatype: string
    """
    # replace links with url-web
    pattern_url = 'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = 'url-web'
    text = re.sub(pattern_url, subs_url, text)
    # replace the html characters with " "
    text = re.sub('<.*?>#()&', ' ', text)
    text = re.sub('&', ' ', text)
#     text = re.sub('(', ' ', text)
#     text = re.sub(')', ' ', text)
    
    text=re.sub('Nan', ' ', text)
    # replace escape sequence with space
    escape_seq = ["\r", "\n",'\a', '\b', '\f', "\`", '\”', '\t']
    for i in escape_seq:
        text = re.sub(i, " ", text)
    # Remove Capitalization
    text = text.lower()
    # plit and join the words
    text=' '.join(text.split())
    
    return text

In [4]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("#",' ',str(text))
    text = re.sub("@",'',str(text))
    
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]#", "", str(text))
    
    return text

In [5]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [6]:
scraped['description'] = scraped['description'].astype(str).apply(clean_text)
scraped['inclusions'] = scraped['inclusions'].astype(str).apply(clean_text)
scraped['hotel_name'] = scraped['hotel_name'].astype(str).apply(clean_text)
scraped['title'] = scraped['title'].astype(str).apply(clean_text)

In [71]:
scraped['corpus'] = ''
corpus = []
columns = ['title','hotel_name','description', 'inclusions']
for i in range(0, len(scraped['scraped_url'])):
    words = ''
    for col in columns:
        words = words + scraped.iloc[i][col] + " "        
    corpus.append(words)
scraped['corpus'] = corpus
#scrapy.set_index('scraped_url', inplace=True)
scraped.drop(columns=['title', 'description', 'inclusions', 'hotel_name','agency_slug','country'], inplace=True)
scraped.head()


TypeError: can only concatenate str (not "float") to str

In [8]:
scraped['corpus'] = scraped['corpus'].astype(str).apply(remove_punctuation)

In [9]:
scraped['corpus'] = scraped['corpus'].astype(str).apply(clean)

In [10]:
hotels['Name'] = hotels['Name'].astype(str).apply(clean_text)

In [11]:
hotels['Name'] = hotels['Name'].astype(str).apply(clean)

In [12]:
hotels['Name'] = hotels['Name'].astype(str).apply(remove_punctuation)

In [13]:
list_hotels=hotels['Name'].astype(str).to_list
type_labels = list(hotels.Name.unique())


In [14]:
type_labels

['41',
 ' 2 andrews \x96 port douglas',
 ' 2 hastings street',
 'com',
 'the dunstane',
 ' beach bed bar',
 ' michigan inn lodge',
 ' samui haus',
 ' the hub west',
 ' white patong',
 'hom hotel kudus by horison group',
 'home boutique hotel',
 'home budget hotel',
 'hua lamphong',
 'inn',
 'm2 villa',
 'thepad hotel resort',
 '06 central hostel buenos aires',
 '08028 apartments',
 '1 baron motel',
 '1 bedroom china town condo',
 '1 borneo tower b service apartment',
 '1 golf place',
 '1 hotel central park',
 '1 hotel south beach',
 '1 lexham gardens hotel',
 '1 london street',
 '1 night 1980 hostel',
 '1 nimman gallery hotel',
 '1 park row b b',
 '1 republic hotel',
 '14 merit montevideo apart suites',
 '13rd residence guesthouse yashiki',
 '13rd residence serviced apartments akasaka',
 '13rd residence serviced apartments akihabara',
 '13rd residence serviced apartments nihonbashi',
 '13rd residence serviced apartments yoyogi',
 '10 2nd avenue houghton estate',
 '10 blossoms hospitali

In [26]:
list_hotels

<bound method IndexOpsMixin.tolist of 0                                41
1          2 andrews  port douglas
2                 2 hastings street
3                               com
4                      the dunstane
                    ...            
145626       zwahili game lodge spa
145627                    zyanrooms
145628          zz house chiang mai
145629           zzziesta chiangmai
145630            zzzip guest house
Name: Name, Length: 145631, dtype: object>

In [35]:
test_list1=type_labels
tel=testt
#res = list(set([ele1 for sub1 in lis for ele1 in lis2 if ele1 in sub1]))
# res = [ele2 for ele2 in test_list2 if any(ele2 in ele1 for ele1 in test_list1)]
res = [ele1 for ele1 in test_list1 if any(ele2 in ele1 for ele2 in tel)]

print(str(res))

['20 at the kings', '5 rooms at the stirling hotel', 'a b b at the edward harris house inn', 'a little house in the colony', 'a place at the beach', 'a place in the sun', 'aarinda holiday apartments', 'acacia beachfront resort', 'accent accommodation at docklands melbourne', 'achinika holiday inn', 'adaaran select hudhuran fushi', 'adaaran select meedhupparu all inclusive', 'affordable suites of america', 'affordable suites of america fredericksburg', 'ahipara holiday park', 'airporthotel berlin adlershof', 'akacfa holiday apartments', 'akamanthea holiday village', 'albion of auckland', 'alden suites  a beachfront resort', 'alice in the territory', 'aloft austin at the domain', 'aloft charlotte uptown at the epicentre', 'aloft detroit at the david whitney', 'amelia hotel at the beach', 'americas best value inn suites at the parkconvention ctr', 'amore at the beach', 'ananda in the himalayas', 'anchorage at the lake', 'angel of the sea', 'angel of the winds casino hotel', 'anglesea beac

In [21]:
from nltk.util import ngrams

In [32]:
def word_grams(words, min_n=2, max_n=4):
    s = []
    for n in range(min_n, max_n):
        for ngram in ngrams(words, n):
            s.append(' '.join(str(i) for i in ngram))
    return s

In [34]:
testt=word_grams(test_list2)

In [44]:
test_list2=['adaaran','mukamijeniffer', 'select', 'meedhupparu', 'nan', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily', 'exclusions', 'items', 'of', 'a', 'personal', 'nature', 'items', 'not', 'mentioned', 'gratuities', 'and', 'porterage', 'meals', 'drinks', 'not', 'mentioned', 'tourssightseeing', 'not', 'mentioned', 'car', 'hire', 'with', 'the', 'promise', 'of', 'an', 'incomparable', 'holiday', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'one', 'of', 'the', 'best', 'maldives', 'luxury', 'resorts', 'offers', 'blissful', 'accommodation', 'for', 'the', 'discerning', 'traveller', 'to', 'the', 'maldives', 'bask', 'under', 'the', 'healing', 'sunshine', 'and', 'soak', 'up', 'the', 'alluring', 'sights', 'and', 'sounds', 'of', 'a', 'truly', 'enchanting', 'island', 'in', 'the', 'tropics', 'adaaran', 'select', 'meedhupparu', 'offers', 'tranquil', 'settings', 'brimming', 'with', 'luxury', 'settle', 'into', 'a', 'holiday', 'full', 'of', 'breathtaking', 'experiences', 'that', 'will', 'leave', 'you', 'with', 'memories', 'to', 'last', 'a', 'lifetime', 'your', 'maldives', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'approximate', 'taxes', 'return', 'domestic', 'flight', 'speedboat', 'transfers', '7', 'nights', 'accommodation', 'at', 'the', '4star', 'adaaran', 'select', 'meedhupparu', 'in', 'a', 'beachfront', 'villa', 'all', 'meals', 'selected', 'alcoholic', 'and', 'nonalcoholic', 'drinks', 'daily']
test_list3=['family', 'fun', 'on', 'the', 'wild', 'coast', 'nan', 'your', 'eastern', 'cape', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'to', 'durban', 'approximate', 'taxes', '3', 'days', 'car', 'hire', 'kia', 'picanto', 'or', 'similar', '3', 'nights', 'midweek', 'accommodation', 'on', 'the', 'wild', 'coast', 'at', 'the', '4star', 'wild', 'coast', 'sun', 'in', 'a', 'garden', 'facing', 'luxury', 'room', 'breakfast', 'daily', 'bonus', 'complimentary', 'access', 'to', 'wild', 'waves', 'water', 'park', 'please', 'note', '2', 'children', 'eat', 'breakfast', 'and', 'stay', 'free', 'when', 'sharing', 'with', 'adults', 'weekend', 'upgrade', 'from', 'r660', 'per', 'adult', 'exclusions', 'transfers', 'items', 'of', 'a', 'personal', 'nature', 'items', 'not', 'mentioned', 'gratuities', 'and', 'porterage', 'meals', 'and', 'drinks', 'not', 'mentioned', 'tourssightseeing', 'not', 'mentioned', 'less', 'than', '2', 'hours', 'from', 'durban', 'you', 'cant', 'beat', 'the', 'location', 'of', 'the', 'wild', 'coast', 'sun', 'hotel', 'situated', 'in', 'the', 'coastal', 'hub', 'of', 'port', 'edward', 'less', 'than', 'a', 'twohour', 'scenic', 'drive', 'along', 'the', 'south', 'coast', 'from', 'durbans', 'king', 'shaka', 'international', 'airport', 'this', 'familyfriendly', 'accommodation', 'is', 'just', 'the', 'place', 'if', 'youre', 'looking', 'for', 'that', 'relaxing', 'room', 'with', 'a', 'view', 'with', 'all', 'the', 'modern', 'facilities', 'one', 'could', 'ever', 'need', 'moms', 'and', 'dads', 'can', 'soak', 'up', 'the', 'sun', 'watching', 'the', 'kids', 'play', 'at', 'the', 'wild', 'waves', 'water', 'park', 'while', 'couples', 'can', 'slip', 'away', 'to', 'the', 'tranquil', 'oasis', 'of', 'the', 'mangwanani', 'african', 'day', 'spa', 'your', 'eastern', 'cape', 'holiday', 'package', 'includes', 'return', 'flights', 'from', 'johannesburg', 'to', 'durban', 'approximate', 'taxes', '3', 'days', 'car', 'hire', 'kia', 'picanto', 'or', 'similar', '3', 'nights', 'midweek', 'accommodation', 'on', 'the', 'wild', 'coast', 'at', 'the', '4star', 'wild', 'coast', 'sun', 'in', 'a', 'garden', 'facing', 'luxury', 'room', 'breakfast', 'daily', 'bonus', 'complimentary', 'access', 'to', 'wild', 'waves', 'water', 'park', 'please', 'note', '2', 'children', 'eat', 'breakfast', 'and', 'stay', 'free', 'when', 'sharing', 'with', 'adults', 'weekend', 'upgrade', 'from', 'r660', 'per', 'adult']

In [36]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
def remove_stop_words(tokens):
    return [t for t in tokens if t not in stopwords.words('english')]


In [45]:
no_stop =remove_stop_words(test_list2)
no_stop3 =remove_stop_words(test_list3)

In [59]:
stop_gram=word_grams(no_stop)
stop_gram3=word_grams(no_stop3)
stop_gram

['adaaran mukamijeniffer',
 'mukamijeniffer select',
 'select meedhupparu',
 'meedhupparu nan',
 'nan maldives',
 'maldives holiday',
 'holiday package',
 'package includes',
 'includes return',
 'return flights',
 'flights johannesburg',
 'johannesburg approximate',
 'approximate taxes',
 'taxes return',
 'return domestic',
 'domestic flight',
 'flight speedboat',
 'speedboat transfers',
 'transfers 7',
 '7 nights',
 'nights accommodation',
 'accommodation 4star',
 '4star adaaran',
 'adaaran select',
 'select meedhupparu',
 'meedhupparu beachfront',
 'beachfront villa',
 'villa meals',
 'meals selected',
 'selected alcoholic',
 'alcoholic nonalcoholic',
 'nonalcoholic drinks',
 'drinks daily',
 'daily exclusions',
 'exclusions items',
 'items personal',
 'personal nature',
 'nature items',
 'items mentioned',
 'mentioned gratuities',
 'gratuities porterage',
 'porterage meals',
 'meals drinks',
 'drinks mentioned',
 'mentioned tourssightseeing',
 'tourssightseeing mentioned',
 'mentio

In [64]:
test_list1=type_labels
tel=stop_gram
#res = list(set([ele1 for sub1 in lis for ele1 in lis2 if ele1 in sub1]))
# res = [ele2 for ele2 in test_list2 if any(ele2 in ele1 for ele1 in test_list1)]
res = [ele1 for ele1 in test_list1 if any(ele2 in ele1 for ele2 in tel)]

print(str(res))
 

['adaaran select hudhuran fushi', 'adaaran select meedhupparu all inclusive', 'beachfront villa mediteran', 'bequia beachfront villa hotel', 'bon azur beachfront villa penthouses', 'fort recovery beachfront villa suites hotel', 'jet luxury resorts  the signature condo hotel', 'jimbaran balangan beachfront villa nakar', 'moana sands beachfront villas', 'paradise fantasy beachfront villa', 'quinta pacifica beachfront villas', 'vale vale beachfront villas']


In [ ]:
df['colF'] = df.apply(lambda row: categorise(row), axis=1)


In [ ]:
def main():

In [53]:
import pycountry

input_countries = ['American Samoa', 'Canada', 'France']

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

codes = [countries.get(country, 'Unknown code') for country in input_countries]

print(codes)  # prints ['AS', 'CA', 'FR']



['AS', 'CA', 'FR']


In [54]:
df = pd.DataFrame({'Country':['Italy', 'Spain', 'China']})

df

,Country
0,Italy
1,Spain
2,China


In [56]:
def findCountry (country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_2
    except:
        return ("not founded")
scraped['country_alpha_2'] = df.apply(lambda row: findCountry(row.Country) , axis = 1)

In [66]:
new_scraped = pd.read_csv('Hotel Location Table Mapping.csv',encoding='latin1')

In [69]:
new_scraped['country_alpha_2'] = scraped.apply(lambda row: findCountry(row.country) , axis = 1)

In [70]:
new_scraped.head()

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,country_alpha_2
0,https://www.flightcentre.co.za/product/14197346,flightcentre,maldives,Adaaran Select Meedhupparu,\nYour Maldives Holiday Package includes:\nRet...,\nYour Maldives Holiday Package includes:\n\nR...,NaN,NaN,MV
1,https://www.flightcentre.co.za/product/5415249,flightcentre,south africa,Family Fun on the Wild Coast,\nYour Eastern Cape Holiday Package includes:\...,\nYour Eastern Cape Holiday Package includes:\...,NaN,NaN,ZA
2,https://www.flightcentre.co.za/product/16385011,flightcentre,mauritius,Azuri Residences by Life in Blue,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN,MU
3,https://www.flightcentre.co.za/product/16323810,flightcentre,mauritius,Seas the Day in Mauritius,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN,MU
4,https://www.flightcentre.co.za/product/13250621,flightcentre,south africa,Kruger Park Splendour,\nYour Kruger National Park Holiday Package in...,\nYour Kruger National Park Holiday Package in...,NaN,NaN,ZA


In [65]:
df

,Country,country_alpha_2
0,Italy,IT
1,Spain,ES
2,China,CN


In [62]:
var=['adaaran select hudhuran fushi', 'adaaran select meedhupparu all inclusive', 'beachfront villa mediteran', 'bequia beachfront villa hotel', 'bon azur beachfront villa penthouses', 'fort recovery beachfront villa suites hotel', 'jet luxury resorts  the signature condo hotel', 'jimbaran balangan beachfront villa nakar', 'moana sands beachfront villas', 'paradise fantasy beachfront villa', 'quinta pacifica beachfront villas', 'vale vale beachfront villas']

In [61]:
var2 ='adaaran select meedhupparu'

In [60]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [63]:
for i in var:
    print(similar(var2,i))

0.7272727272727273
0.7878787878787878
0.19230769230769232
0.21818181818181817
0.12903225806451613
0.17391304347826086
0.14084507042253522
0.3333333333333333
0.32727272727272727
0.3389830508474576
0.2711864406779661
0.2641509433962264
